In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *
!pip install fastai --upgrade
from fastai.vision.core import *
from fastai.vision.data import *

In [ ]:
df= pd.read_csv("/kaggle/input/noaa-right-whale-recognition/sample_submission.csv")
df

In [ ]:
df2= pd.read_csv("/kaggle/input/noaa-right-whale-recognition/train.csv")
df2.head()

In [ ]:
import fastai
import fastai.vision.all

In [ ]:
df1=pd.read_csv("/kaggle/input/noaa-right-whale-recognition/train.csv")
print(df1.head())
print("total number of whale classes is", len(df1.whaleID.unique()))

In [ ]:
!ls "/kaggle/working/imgs" -aU| head -3

In [ ]:
#df1['location']=""
#train_list=[]
#count=0
#for i in os.listdir("/kaggle/working/imgs"):
 #   df1.loc[df1['Image']==i,'location']="/kaggle/working/imgs/"+i

In [ ]:
import PIL 
#PIL.Image.open("/kaggle/working/imgs/w_6365.jpg")
#PIL.Image.open(df1.iloc[0,2])

In [ ]:
df1.head(3)

In [ ]:
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.vision.data import *
comp = 'noaa-right-whale-recognition'
data_dir = setup_comp(comp)
imgs_dir = untar_dir(data_dir/'imgs.zip', Path('imgs'))
     


In [ ]:
img_files=get_image_files(imgs_dir)

In [ ]:
def plot_sizes(image_sizes, max_n=1000):
    sizes = parallel(image_size, img_files[:max_n], progress=progress_bar)
    
    widths, heights = zip(*sizes)
    min_x = min(widths + heights)
    max_x = max(widths + heights)
    plt.hist(widths, label='width', range=(min_x, max_x))
    plt.hist(heights, label='height', range=(min_x, max_x))
    plt.legend();

plot_sizes(img_files)


In [ ]:
!rm -rf /kaggle/working/imgs_480max

In [ ]:
if not Path('imgs_480max').exists():
    resize_images(imgs_dir, dest='imgs_480max', max_size=480, progress=progress_bar)

In [ ]:
image_files=get_image_files('imgs_480max')
plot_sizes(image_files)

In [ ]:
targs_df=pd.read_csv(data_dir/'train.csv')
targs_df=targs_df[targs_df['Image']!='w_7489.jpg']
items=targs_df['Image']

In [ ]:
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.vision.data import *
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files,#df1["Image"].all(), 
                 splitter=RandomSplitter(),
                 get_y=df1['whaleID'].all(),
                 item_tfms=Resize(128),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))

In [ ]:
def get_image_path(name):
    return Path('imgs_480max')/name
def get_label(name):
    return targs_df[targs_df["Image"]==name]['whaleID'].item()


In [ ]:
x_pipe = [get_image_path, PILImage.create]
y_pipe = [get_label, Categorize()]

In [ ]:
# make sure at least one of each whale is in training set, then randomly split
must_train_whales = targs_df.groupby('whaleID').first()['Image']
# some magic from [here](https://stackoverflow.com/questions/49823963/get-index-of-one-series-into-another-in-pandas)
must_train_ids = pd.Series(targs_df['Image'].index, index=targs_df['Image']).get(must_train_whales)
must_train_ids = set(must_train_ids)

train_ids, valid_ids = RandomSplitter(seed=42)(items)
print(f"Before: train_ids={len(train_ids)}, valid_ids={len(valid_ids)}")
train_ids = L(set(train_ids).union(must_train_ids))
valid_ids = L(set(valid_ids) - must_train_ids)
print(f"After: train_ids={len(train_ids)}, valid_ids={len(valid_ids)}")
splits = (train_ids, valid_ids)

In [ ]:
items

In [ ]:
dss = Datasets(items, [x_pipe, y_pipe], splits=splits)

In [ ]:
items


In [ ]:
dss.show(dss[105])

In [ ]:
after_item = [ToTensor(), Resize((320, 480))]
after_batch = [IntToFloatTensor(), *aug_transforms(size=(224, 336))]

dls = dss.dataloaders(32, after_item=after_item, after_batch=after_batch)

In [ ]:
dls.show_batch()

In [ ]:
metrics=[error_rate]
learn=vision_learner(dls,'resnet26d',metrics=metrics).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:


learn.fine_tune(10, 0.002)
